In [1]:
#RDD创建   入口对象
from pyspark import SparkConf,SparkContext

conf=SparkConf().setAppName('spark2').setMaster('local[*]')

sc=SparkContext(conf=conf)

22/05/10 13:57:00 WARN Utils: Your hostname, MaQiang resolves to a loopback address: 127.0.1.1; using 192.168.1.110 instead (on interface wlp2s0)
22/05/10 13:57:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/10 13:57:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## RDD两种创建方式 : 并行化,文件创建

### parallelize

In [2]:
#并行化创建RDD,默认8 个分区(与CPU有关)
rdd =sc.parallelize([1,2,3,4,5,7,8,9,10])

#查看分区数
rdd.getNumPartitions()

8

In [3]:
#分布式 转换 为 list
rdd.collect()

[1, 2, 3, 4, 5, 7, 8, 9, 10]

### textFile

In [4]:
#默认分区数量根文件大小有关
rdd=sc.textFile('txt/spark2.txt')

In [5]:
rdd.getNumPartitions()

2

In [6]:
rdd.collect()

['hello hello txt txt',
 'test test',
 'test qiang qiang',
 'hello hadoop spark spark']

##### 小文件多用 wholeTextFile

In [7]:
rdd = sc.wholeTextFiles("txt")
rdd.getNumPartitions()

1

In [8]:
rdd.map(lambda x:x[1]).collect()

['maqiang maqiang maqiang',
 'hello hello txt txt\ntest test\ntest qiang qiang\nhello hadoop spark spark']

## rdd算子有两类: transformation 和 Action 
##### 返回RDD的是transf
##### collection 这种 不返回RDD 是action

### map 

对每个分区相同的操作

In [9]:
#传入方法
def add(a):
    return a+100

rdd=sc.parallelize([1,2,3,4,5,6,78,9,0,123]).map(add)
rdd.collect()

[101, 102, 103, 104, 105, 106, 178, 109, 100, 223]

In [10]:
# lambda

rdd=sc.parallelize([1,2,3,4,5,6,78,9,0,123]).map(lambda x: x+100)
rdd.collect()

[101, 102, 103, 104, 105, 106, 178, 109, 100, 223]

### flatMap
list解除子嵌套,然后map

In [11]:
rdd=sc.textFile('txt/spark2.txt')
rdd1=rdd.flatMap(lambda x: x.split(" "))

In [12]:
rdd1.collect()

['hello',
 'hello',
 'txt',
 'txt',
 'test',
 'test',
 'test',
 'qiang',
 'qiang',
 'hello',
 'hadoop',
 'spark',
 'spark']

### reduceByKey
根据 key 计算

In [13]:
rdd2=rdd1.map(lambda x: (x,1))

#累加
rdd3=rdd2.reduceByKey(lambda x,y:x+y)
rdd3.collect()


[('test', 3),
 ('qiang', 2),
 ('hadoop', 1),
 ('hello', 3),
 ('txt', 2),
 ('spark', 2)]

### mapValues 只针对value操作

In [14]:
rdd2.map(lambda x:(x[0],x[1]*10)).collect()

[('hello', 10),
 ('hello', 10),
 ('txt', 10),
 ('txt', 10),
 ('test', 10),
 ('test', 10),
 ('test', 10),
 ('qiang', 10),
 ('qiang', 10),
 ('hello', 10),
 ('hadoop', 10),
 ('spark', 10),
 ('spark', 10)]

In [15]:
#等价
rdd2.mapValues(lambda x:x*10).collect()

[('hello', 10),
 ('hello', 10),
 ('txt', 10),
 ('txt', 10),
 ('test', 10),
 ('test', 10),
 ('test', 10),
 ('qiang', 10),
 ('qiang', 10),
 ('hello', 10),
 ('hadoop', 10),
 ('spark', 10),
 ('spark', 10)]

## world count

In [16]:
rdd=sc.textFile("txt/spark2.txt")
rdd1=rdd.flatMap(lambda x: x.split(" "))
rdd2=rdd1.map(lambda x:(x,1))

In [17]:
rdd3=rdd2.reduceByKey(lambda x,y:x+y)

In [18]:
rdd3.collect()

[('test', 3),
 ('qiang', 2),
 ('hadoop', 1),
 ('hello', 3),
 ('txt', 2),
 ('spark', 2)]

### groupBy
hash

In [19]:
rdd2.collect()

[('hello', 1),
 ('hello', 1),
 ('txt', 1),
 ('txt', 1),
 ('test', 1),
 ('test', 1),
 ('test', 1),
 ('qiang', 1),
 ('qiang', 1),
 ('hello', 1),
 ('hadoop', 1),
 ('spark', 1),
 ('spark', 1)]

In [20]:
rdd2.groupBy(lambda x:x[0]).collect()

[('test', <pyspark.resultiterable.ResultIterable at 0x7fb24d2ef2e0>),
 ('qiang', <pyspark.resultiterable.ResultIterable at 0x7fb24d31d370>),
 ('hadoop', <pyspark.resultiterable.ResultIterable at 0x7fb24d31d3d0>),
 ('hello', <pyspark.resultiterable.ResultIterable at 0x7fb24d31d430>),
 ('txt', <pyspark.resultiterable.ResultIterable at 0x7fb24d31d490>),
 ('spark', <pyspark.resultiterable.ResultIterable at 0x7fb24d31d250>)]

In [21]:
rdd2.groupBy(lambda x:x[0]).map(lambda x: (x[0],list(x[1]))).collect()

[('test', [('test', 1), ('test', 1), ('test', 1)]),
 ('qiang', [('qiang', 1), ('qiang', 1)]),
 ('hadoop', [('hadoop', 1)]),
 ('hello', [('hello', 1), ('hello', 1), ('hello', 1)]),
 ('txt', [('txt', 1), ('txt', 1)]),
 ('spark', [('spark', 1), ('spark', 1)])]

### filter 保留想要的

In [22]:
rdd3.collect()

[('test', 3),
 ('qiang', 2),
 ('hadoop', 1),
 ('hello', 3),
 ('txt', 2),
 ('spark', 2)]

In [23]:
rdd3.filter(lambda x: x[1]>2).collect()

[('test', 3), ('hello', 3)]

### distinct
去重

In [24]:
rdd2.collect()

[('hello', 1),
 ('hello', 1),
 ('txt', 1),
 ('txt', 1),
 ('test', 1),
 ('test', 1),
 ('test', 1),
 ('qiang', 1),
 ('qiang', 1),
 ('hello', 1),
 ('hadoop', 1),
 ('spark', 1),
 ('spark', 1)]

In [25]:
rdd2.distinct().collect()

[('hello', 1),
 ('txt', 1),
 ('spark', 1),
 ('test', 1),
 ('qiang', 1),
 ('hadoop', 1)]

### Union
合并 

In [26]:
rdd2.union(rdd3).collect()

[('hello', 1),
 ('hello', 1),
 ('txt', 1),
 ('txt', 1),
 ('test', 1),
 ('test', 1),
 ('test', 1),
 ('qiang', 1),
 ('qiang', 1),
 ('hello', 1),
 ('hadoop', 1),
 ('spark', 1),
 ('spark', 1),
 ('test', 3),
 ('qiang', 2),
 ('hadoop', 1),
 ('hello', 3),
 ('txt', 2),
 ('spark', 2)]

### join

In [27]:
rdda=sc.parallelize([(1001,'qiang'),(1002,'ma'),(1003,'ferro')])
rddb=sc.parallelize([(1001,'unipi'),(1003,'uniupo'),(1002,'polito')])

In [28]:
#根据key关联
rdda.join(rddb).collect()

[(1001, ('qiang', 'unipi')),
 (1002, ('ma', 'polito')),
 (1003, ('ferro', 'uniupo'))]

In [29]:
#left
rdda=sc.parallelize([(1001,'qiang'),(1002,'ma'),(1003,'ferro')])
rddb=sc.parallelize([(1001,'unipi'),(1003,'uniupo')])
rdda.leftOuterJoin(rddb).collect()

[(1001, ('qiang', 'unipi')), (1002, ('ma', None)), (1003, ('ferro', 'uniupo'))]

In [30]:
#right
rdda=sc.parallelize([(1001,'qiang'),(1002,'ma'),(1003,'ferro')])
rddb=sc.parallelize([(1001,'unipi'),(1003,'uniupo')])
rdda.rightOuterJoin(rddb).collect()

[(1001, ('qiang', 'unipi')), (1003, ('ferro', 'uniupo'))]

### intersecton
交集

In [31]:
rdd2.collect()

[('hello', 1),
 ('hello', 1),
 ('txt', 1),
 ('txt', 1),
 ('test', 1),
 ('test', 1),
 ('test', 1),
 ('qiang', 1),
 ('qiang', 1),
 ('hello', 1),
 ('hadoop', 1),
 ('spark', 1),
 ('spark', 1)]

In [32]:
rdd3.collect()

[('test', 3),
 ('qiang', 2),
 ('hadoop', 1),
 ('hello', 3),
 ('txt', 2),
 ('spark', 2)]

In [33]:
rdd2.intersection(rdd3).collect()

[('hadoop', 1)]

### glom 根据分区嵌套

glom 与 flatmap相反

In [34]:
rdd2.glom().collect()

[[('hello', 1),
  ('hello', 1),
  ('txt', 1),
  ('txt', 1),
  ('test', 1),
  ('test', 1),
  ('test', 1),
  ('qiang', 1),
  ('qiang', 1)],
 [('hello', 1), ('hadoop', 1), ('spark', 1), ('spark', 1)]]

In [35]:
rdd2.glom().flatMap(lambda x:x).collect()

[('hello', 1),
 ('hello', 1),
 ('txt', 1),
 ('txt', 1),
 ('test', 1),
 ('test', 1),
 ('test', 1),
 ('qiang', 1),
 ('qiang', 1),
 ('hello', 1),
 ('hadoop', 1),
 ('spark', 1),
 ('spark', 1)]

### groupByKey

In [36]:
rdd2.collect()

[('hello', 1),
 ('hello', 1),
 ('txt', 1),
 ('txt', 1),
 ('test', 1),
 ('test', 1),
 ('test', 1),
 ('qiang', 1),
 ('qiang', 1),
 ('hello', 1),
 ('hadoop', 1),
 ('spark', 1),
 ('spark', 1)]

In [37]:
rdd2.groupByKey().map(lambda x:(x[0],list(x[1]))).collect()

[('test', [1, 1, 1]),
 ('qiang', [1, 1]),
 ('hadoop', [1]),
 ('hello', [1, 1, 1]),
 ('txt', [1, 1]),
 ('spark', [1, 1])]

### sortBy

In [44]:
rdd3.sortBy(lambda x:x[1]).collect()

[('hadoop', 1),
 ('qiang', 2),
 ('txt', 2),
 ('spark', 2),
 ('test', 3),
 ('hello', 3)]

In [43]:
rdd3.sortBy(lambda x:x[1],False).collect()

[('test', 3),
 ('hello', 3),
 ('qiang', 2),
 ('txt', 2),
 ('spark', 2),
 ('hadoop', 1)]

In [47]:
#根据key字母排序
rdd3.sortByKey().collect()

[('hadoop', 1),
 ('hello', 3),
 ('qiang', 2),
 ('spark', 2),
 ('test', 3),
 ('txt', 2)]